In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

X_train = pd.read_csv('/kaggle/input/moa-1111a-enrich-source/moa_1111A_X_train.zip')
X_test = pd.read_csv('/kaggle/input/moa-1111a-enrich-source/moa_1111A_X_test.zip')
Y_train = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
X_train.shape, X_test.shape, X_train.shape

# splitting the data
from sklearn.model_selection import train_test_split
X_train_tr, X_train_te, Y_train_tr, Y_train_te = train_test_split(X_train,Y_train,test_size=0.3,random_state=101)

# Using Keras 
# -- Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import layers


cols_output = Y_train_tr.columns[1:] # first column is sig_id
X_tr = X_train_tr.drop('sig_id',axis=1)
Y_tr = Y_train_tr[cols_output]
X_te = X_train_te.drop('sig_id',axis=1)
Y_te = Y_train_te[cols_output]

inputs = Input(shape=(X_tr.shape[1],))
bn = BatchNormalization()(inputs)
dp=Dropout(0.2, name='dp1')(bn)
hidden = Dense(2048, activation='relu', name='ly1')(dp)
#wn = WeightNormalization()(hidden) # not present in keras
bn = BatchNormalization()(hidden)
dp = Dropout(0.5, name='dp2')(bn)

hidden = Dense(1048, activation='relu', name='ly2')(dp)
#wn = WeightNormalization()(hidden)
bn = BatchNormalization()(hidden)
dp = Dropout(0.5, name='dp3')(bn)

out = Dense(206, activation='sigmoid', name='ly3')(dp)

#model = Model(inputs=inputs, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9,out10])
model = Model(inputs=inputs, outputs=out)

print(model.summary())

#--- Run Keras Model
losses = 'binary_crossentropy'
#model.compile(optimizer = "sgd", loss = 'mse')
model.compile(loss=losses, optimizer="adam", metrics=['accuracy'])
history = model.fit(X_tr, Y_tr, epochs=100, batch_size=128, validation_data=(X_te, Y_te.values))  # batch_size=128 was critical to get the high accuracy
# add the patters
#df_submission[X_test.cp_type == 1][df_submission.columns[1:]] = 0

# preparing for submission
Y_test_pred = model.predict(X_test[X_test.columns[1:]])
# df_submission = pd.DataFrame(X_test.sig_id)
# df_submission = pd.concat([df_submission,pd.DataFrame(Y_test_pred)], axis=1)
# df_submission.columns = Y_train.columns
# # implement a pattern
# df_submission.loc[X_test['cp_type'] == 1, Y_train.columns[1:]] = 0
# #
# df_submission.to_csv('submission.csv', index=False)
# df_submission.shape

# # Validation checks of submission with sample submission
# sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
# print(set(df_submission.dtypes == sample_submission.dtypes)) # check if ID column type is identical
# print(set(df_submission['sig_id'] == sample_submission['sig_id'])) # check if ID rows are identical
# print(df_submission.shape, sample_submission.shape) # check if shapes are identical
# print(set(df_submission.columns == sample_submission.columns)) # check if columns are identical

# print(df_submission.dtypes)
# print(sample_submission.dtypes)

In [ ]:
df_submission.head()

In [ ]:
sam = pd.read_csv('../input/lish-moa/sample_submission.csv')
sam.head()

In [ ]:
# df_submission.columns

In [ ]:
sub = model.predict(X_test[X_test.columns[1:]])

In [ ]:
sam.iloc[:,1:]=sub

In [ ]:
sam.to_csv('submission.csv',index=False)